<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
<h1> This notebook walks us through how to compute a tuning curve  </h1>
    
A tuning curve summarizes a neuron's response to stimulus categories. Here we will look at the responses of a neuron to the 'Drifting Gratings' stimulus.

The Drifting Gratings stimulus consists of a sinusoidal grating that moves in 8 directions and at 5 temporal frequencies. Each stimulus condition (direction + temporal frequency combination) is repeated 15 times in random order. In addition, there are blanksweeps (when the grating is replaced with mean luminance gray) interleaved among the trials.

We will compute the mean response of a neuron to each of these stimulus conditions.

The data we will use are from the Allen Brain Observatory.  We have taken some data for a specific cell and saved it in the directory `data`.  We will not discuss general mechanisms of accessing Brain Observatory data here.  (For future reference, we will look at responses from the cell with `cell_id` 541513979.)
</div>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
In order to compute the tuning curve of the neuron, we need
<li>the activity of the neuron.  We will use the DF/F trace
<li>stimulus information for the drifting grating stimulus
 
We have stored the activity information in the arrays loaded with the following commands.

`dff_trace` contains the activity.  The index for this array is the acquisition frame number of the calcium recording.  Each acquisition frame corresonds to the clock time given in the `timestamps` array.  

</div>

In [ ]:
timestamps = np.load('data/timestamps.npy')
dff_trace = np.load('data/dff_trace.npy')

<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
It is always good to look at the data objects you're working with to make sure you understand what they are. What is the shape of the dff_trace array?

In [ ]:
print(dff_trace.shape)
print(timestamps.shape)

<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
Let's plot the DF/F trace of our neuron to see what it looks like

In [ ]:
fig, ax = plt.subplots(figsize=(14,5))
ax.plot(timestamps, dff_trace)
ax.set_xlabel("Time (s)", fontsize=16)
ax.set_ylabel("DF/F", fontsize=16)

<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
These are just the responses over the course of the experiment.  Let's look at what stimuli were used and when they were displayed.

In [ ]:
stimulus = np.load('data/stimulus.npy', allow_pickle=True)
epoch_frames = np.load('data/epoch_frames.npy')

In [ ]:
stimulus

<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
The `epoch_frames` array contains the acquisition frames at which each stimulus type starts and ends.  The first column is the start, the second column is the end.

In [ ]:
print(epoch_frames)

<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
Let's see how these correspond to each other.  We'll make use of the `enumerate` function.

In [ ]:
for i, stim_name in enumerate(stimulus):
    print(i, ':  ', stim_name)

In [ ]:
for i, stim_name in enumerate(stimulus):
    print(stim_name, ':\t', epoch_frames[i][0], ', ', epoch_frames[i][1])

In [ ]:
fig, ax = plt.subplots(figsize=(14,5))
ax.plot(timestamps, dff_trace)
ax.set_xlabel("Time (s)", fontsize=16)
ax.set_ylabel("DFF", fontsize=16)

start, end = epoch_frames[0]

ax.axvspan(timestamps[start], timestamps[end], color='red',alpha=0.3)

<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
Let's define a dictionary to map color values to stimulus names.  We'll use this in the exercise below.

In [ ]:
color_dict = {'drifting_gratings':  'red',
              'natural_movie_three': 'blue',
              'natural_movie_one': 'purple',
              'spontaneous': 'grey'}

**Exercise 1**:  Plot the DF/F trace of this neuron.  Shade the regions that correspond to each stimulus type shown using the colors defined in `color_dict` above.  (Hint:  Use the method `axvspan`, which can be called with either `plt.axvspan` or `ax.axvspan`.  Also, be aware that we are plotting versus the *timestamp*, not the acquisition frame.)

<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
It appears that this cell responds quite well to the drifting gratings stimulus.  We've stored the information for how the drifting gratings were displayed in the arrays below.  Each presentation is called a "trial".  For each trial, there is an direction and a temporal frequency for the grating.  Some trials correspond to blank sweeps when nothing is on the screen.  The `frames` array contains the start and end times for the display of that trial, similarly to the `epoch_frames` array.

In [ ]:
temporal_frequency = np.load('data/temporal_frequency.npy')
direction = np.load('data/direction.npy')
blank_sweep = np.load('data/blank_sweep.npy')
frames = np.load('data/frames.npy')

<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
Let's look at the direction array.

In [ ]:
direction

<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
How many unique temporal frequencies and directions are there in this stimulus? Let's print the unique values for each of these parameters.

In [ ]:
np.unique(direction)

In [ ]:
np.unique(temporal_frequency)

In [ ]:
frames

<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
    To look at the cell's response to a given grating presentation, let's plot the DF/F of the cell during the presentation of that grating.  We want to pad the plot with ~ 1 second of the DF/F trace preceding the grating presentation and ~1 after.  1 second = 30 frames.  We'll plot the response to the first grating presentation.

In [ ]:
start, end = frames[0]

fig, ax = plt.subplots()
ax.plot(dff_trace[start-30:end+30])
ax.axvspan(30,30+end-start, color='gray', alpha=0.3) #this shades the period when the stimulus is being presented
ax.set_ylabel("DF/F")
ax.set_xlabel("Frames")

<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
    We want to quantify this response. There are different methods of quantifying this that you can explore:
<li> mean DF/F during the grating presentation
<li> sum of the DF/F during the grating presentation (are these different?)
<li> maximum DF/F during grating</li>
<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
Do you have other ideas for how to quantify this response? 
    
   

<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
   For now let's use the mean DF/F during the presentation of the grating.  

In [ ]:
dff_trace[start:end].mean()

**Exercise 2:** Repeat this for the next grating stimulus: plot the next trial and calculate the mean DF/F.

<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
Already we can see that some stimulus conditions elicit larger responses than others.  This is what we want to quantify and summarize in a <b>tuning curve</b>.
<p>To do this, let's calculate the mean DF/F for each grating presentation in this stimulus. To start, let's create a numpy array to hold our calculated responses for all of the trials. We'll have three columns, one for the stimulus direction, one for the stimulus temporal frequency, and the last for the cell's response. Then we need to iterate over all stimulus trials, populate the direction and TF and then calculate the mean response.
</div>

**Exercise 3**:  Create a numpy array containing the direction, temporal_frequency, and mean response for each trial.  

In [ ]:
num_trials = len(direction)

cell_response= np.zeros((num_trials,3))

for i in range(num_trials):
    # fill in code here

In [ ]:
cell_response

<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
If we only care about one stimulus parameter, we can quickly compare the response to that parameter, say the direction. Here we will plot each grating response as a function of the grating direction

**Exercise 4a**:  Plot the DF/F response for each trial against the direction for the drifting grating on that trial.

<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
What do we see?
</div>

**Exercise 4b:** Repeat this for the temporal frequency parameter

<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
We want to quantify this more explicitly.  So let's average all of the responses to each direction together. This is the mean DF/F response to an direction, for all temporal frequencies, for all trials.  To do this, we need a way of selecting the trials that correspond to a given direction  For example, for direction=270:

In [ ]:
#Find the trials where the direction is 270
trial_mask = cell_response[:,0]==270

trial_mask

In [ ]:
#the mean DF/F of just those trials together
cell_response[trial_mask,2]

In [ ]:
#Average the mean DF/F of just those trials together
print("The mean response over all trials in response to stimuli with direction=270:")
cell_response[trial_mask,2].mean()

<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
Let's compute and plot the mean response as a function of direction (averaged across all temporal frequencies).   
<p> To start, we need to know what all the possible direction values are. We need to identify the <b>unique</b> values that are not NaNs (e.g. values that are <b>finite</b>)

In [ ]:
all_dir = np.unique(cell_response[:,0])
print(all_dir)
dirvals = all_dir[np.isfinite(all_dir)]
print(dirvals)

<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
Now let's make an array with the average response of all trials for each of these directions.  We'll use `enumerate` again. 

**Exercise 5a**:  Compute and plot the tuning curve as a function of direction (averaged over temporal frequencies).  

Hint: start by creating an array with length equal to the number of directions, then use a for loop to compute the average response to each direction. 

In [ ]:
tuning = np.empty((8))

for i, dirv in enumerate(dirvals):
    # fill in code here

<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
Let's plot this tuning curve of mean response vs direction.

**Exercise 5b:** Compute and plot the mean response as a function of temporal frequency (averaged across all directions).

<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
Compare these curves with the plots we made above of all the trials. What do we see now?
</div>

**Exercise 6:** Add errorbars to the above tuning curves. They can be standard deviation or standard error or the mean. 

Make a new array for the tuning values with two columns - one for the mean response and one for the error you decide to use.  (Hint:  the standard deviation can be computed with a method call `std` in the same way you compute the mean with a method called `mean`.)
    
<p>(Hint: for plotting, <b>plt.errorbar</b> will be useful).

In [ ]:
? plt.errorbar

In [ ]:
tuning_dir = np.empty((8,2))
for i, dirv in enumerate(dirvals):
    # fill in code here

In [ ]:
tuning_tf = np.empty((5,2))
for i, tf in enumerate(tfvals):
    # fill in code here

**Exercise 7a:** Add a black line showing the mean response to the blank sweep.
<br>(Hint 1: You can use the `blank_sweep` array to find the trials with a blank sweep.  Hint 2: <b>plt.axhline</b> is a useful function for adding a horizontal line).

In [ ]:
blank_response = 

Why is it important to add these two features?

**Exercise 7b:** Add the errorbars and blank sweep response to the temporal frequency plot as well

<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
So far we've looked at one stimulus dimension (e.g. direction) averaged across all conditions of the other (e.g. temporal frequency) - and vice versa. Now we want to look at these tuning curves for both dimensions.
<p>To begin, let's compute the two-dimensional tuning array for this neuron - computing the mean response for every possible direction and temporal frequency combination.
<p>What is the shape of the array we will be computing?

In [ ]:
tuning_array = np.empty((8,5))


for i,tf in enumerate(tfvals):
    #select trials of each tf
    tf_trials = cell_response[:,1]==tf
    subset = cell_response[tf_trials]
    for j,dirv in enumerate(dirvals):
        #select trials of that tf and each direction
        trials = subset[:,0]==dirv
        tuning_array[j,i] = subset[trials,2].mean()

<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
Let's start by visualizing this similarly to how we did above: Plot the direction tuning curve for each temporal frequency value as a separate line:

In [ ]:
fig, ax = plt.subplots()

for i, tf in enumerate(tfvals):
    ax.plot(dirvals, tuning_array[:,i], 'o-', label=tf)
ax.legend()
ax.set_xlabel("Direction (deg)")
ax.set_ylabel("Mean DF/F")


**Exercise 9:** Plot each temporal frequency tuning curve for each direction as a separate line.

<div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
What do we see here?
<p>Another way to visualize two dimensional data is as a heatmap. Let's use <b>plt.imshow</b> to create this heatmap.</p>

In [ ]:
fig, ax = plt.subplots()

im = ax.imshow(tuning_array)
ax.set_xticks(range(5), tfvals)
ax.set_yticks(range(8), dirvals)
ax.set_xlabel("TF")
ax.set_ylabel("Direction")
cbar = plt.colorbar(im) #Add the colorbar so we know what the colors mean
cbar.set_label("Mean DF/F")

 <div style="background: #F0FAFF; border-radius: 3px; padding: 10px;">
<p>This visualization gives us a great view of how the two dimensions interact, but one disadvantage is that we can't add errorbars or even the blank sweep response to this. But we saw that that information can be important for interpreting what we see in the data.
<p>One approach we can take is to compute the <b>z score</b> where each response is mean subtracted and normalized to the standard deviation. So it shows how far each response deviates from the mean response (rather than the blanksweep response as we plotted above in exercise 5).

In [ ]:
tuning_array_z = (tuning_array - tuning_array.mean())/tuning_array.std()

In [ ]:
fig, ax = plt.subplots()

im = ax.imshow(tuning_array_z)
cbar = plt.colorbar(im) #Add the colorbar so we know what the colors mean
cbar.set_label("Z score (std dev)")